Examples using STAC with Intake
================

In this notebook, we'll take a look at some of the functionality in Intake-STAC by exploring the [Planet Disaster data catalog](https://planet.stac.cloud/?t=catalogs).

In [ ]:
%matplotlib inline

from intake_stac import STACCatalog

## Catalog adapter

We'll start by connecting to the STAC catalog for `planet-disaster-data`.

In [ ]:
cat = STACCatalog('https://storage.googleapis.com/pdd-stac/disasters/catalog.json', 'planet-disaster-data')
display(list(cat))

## GeoTIFF --> `xarray.DataArray`

Using the intake-xarray plugin and xarray's rasterio integration, we can open GeoTIFF datasets remotely. 

In [ ]:
tif = cat['Houston-East-20170831-103f-100d-0f4f-RGBthumbnail']
tiff_da = tif.to_dask()
tiff_da

In [ ]:
tiff_da.plot.imshow(rgb='channel')